In [1]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

# import findspark
# findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("RFM Analysis with PySpark") \
    .getOrCreate()

In [4]:
import pandas as pd
x_data = pd.read_csv('C:/Users/Gajraj Singh/Desktop/LTV1-Namashi.csv')
#spark code for read_csv
# data = spark.read.format("csv").option("header", "true").load("C:/Users/Gajraj Singh/Desktop/LTV1-Namashi.csv")

In [5]:
x_data.head()

,InvoiceDate,isattr,eventItemJson
0,2019-10-18,False,"{""locale"":""ar_SA"",""criteo_p"":""%5B%7B%22i%22:%2..."
1,2019-10-18,False,"{""device_name"":""iPhone"",""locale"":""en_AE"",""fb_c..."
2,2019-10-18,False,"{""revenue"":""27"",""fb_value"":""27"",""device_name"":..."
3,2019-10-18,False,"{""transaction_id"":""SA7A7066474"",""fb_currency"":..."
4,2019-10-18,False,"{""fb_content"":""[{\""id\"":\""GL103AT84LOD\"",\""qua..."


In [6]:
import json
x_data['InvoiceNo'] = x_data['eventItemJson'].apply(lambda x:json.loads(x).get('transaction_id','NULL'))
x_data['StockCode'] = x_data['eventItemJson'].apply(lambda x:json.loads(x).get('g_sku','NULL'))
x_data['CustomerID'] = x_data['eventItemJson'].apply(lambda x:json.loads(x).get('user_id','NULL'))
x_data['Revenue'] = x_data['eventItemJson'].apply(lambda x:json.loads(x).get('revenue','NULL'))

In [7]:
x_data = x_data.drop(['eventItemJson'],axis=1)

In [8]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101897 entries, 0 to 1101896
Data columns (total 6 columns):
InvoiceDate    1101897 non-null object
isattr         1101897 non-null bool
InvoiceNo      1101897 non-null object
StockCode      1101897 non-null object
CustomerID     1101897 non-null object
Revenue        1101897 non-null object
dtypes: bool(1), object(5)
memory usage: 43.1+ MB


In [9]:
# %pyspark
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
data =sqlCtx.createDataFrame(x_data)

In [10]:
data.printSchema()

root
 |-- InvoiceDate: string (nullable = true)
 |-- isattr: boolean (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Revenue: string (nullable = true)



In [11]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
data = data.withColumn("Revenue", data["Revenue"].cast(DoubleType()))
# data = data.withColumn("InvoiceDate", to_date(unix_timestamp("InvoiceDate", "MM/dd/yyyy").cast("timestamp")))
data.select('InvoiceDate',from_unixtime(unix_timestamp('InvoiceDate', '%M/%d/%y')).alias('InvoiceDate'))


# define Total column
# data = data.withColumn("Total", round(data["UnitPrice"] * data["Quantity"], 2))


# calculate difference in days between 2011-12-31 and the Invoice Date
data = data.withColumn("RecencyDays", expr("datediff('2020-01-15', InvoiceDate)"))

In [12]:
data.show(5)

+-----------+------+-----------+--------------------+----------+-------+-----------+
|InvoiceDate|isattr|  InvoiceNo|           StockCode|CustomerID|Revenue|RecencyDays|
+-----------+------+-----------+--------------------+----------+-------+-----------+
| 2019-10-18| false|SA7A7882559|   "72704SH52LQP-9S"|   5982454|  121.0|         89|
| 2019-10-18| false|AE7A9279439|"SW493AC30KBX-929...|   7476848|  104.0|         89|
| 2019-10-18| false|AE7A5386753|"CA787AT17WJI-152...|   2702676|   27.0|         89|
| 2019-10-18| false|SA7A7066474|"FO436AC46PJL-150...|   5072903|  110.0|         89|
| 2019-10-18| false|AE7A1635365|"GL103AT84LOD-107...|   5571728|   16.0|         89|
+-----------+------+-----------+--------------------+----------+-------+-----------+
only showing top 5 rows



In [52]:
rfm_table = data.groupBy("CustomerId").agg(min("RecencyDays").alias("Recency"),\
                                           count("InvoiceNo").alias("Frequency"),sum("Revenue").alias("Monetary"))

In [91]:
# rfm_table.count()
rfm_table.orderBy(desc("Frequency")).take(2)
# rfm_table.agg

[Row(CustomerId='NULL', Recency=0, Frequency=110431, Monetary=12712381.0),
 Row(CustomerId='2859680', Recency=0, Frequency=191, Monetary=32938.0)]

In [43]:
rfm_table = rfm_table.withColumn("F", \
                                 when(col("Monetary") >= F[2] , 4).\
                                 when(col("Monetary") >= F[1] , 3).\
                                 when(col("Monetary") >= F[0] , 2).\
                                 otherwise(1))



# R = rfm_table.approxQuantile("Recency", [0.25, 0.5, 0.75], 0)
# F = rfm_table.approxQuantile("Frequency", [0.25, 0.5, 0.75], 0)
# M = rfm_table.approxQuantile("Monetary", [0.25, 0.5, 0.75], 0)

# rfm_table = rfm_table.withColumn("R", \
#                                  when(col("Recency") >= R[2] , 1).\
#                                  when(col("Recency") >= R[1] , 2).\
#                                  when(col("Recency") >= R[0] , 3).\
#                                  otherwise(4))

# rfm_table = rfm_table.withColumn("F", \
#                                  when(col("Monetary") >= F[2] , 4).\
#                                  when(col("Monetary") >= F[1] , 3).\
#                                  when(col("Monetary") >= F[0] , 2).\
#                                  otherwise(1))

# rfm_table = rfm_table.withColumn("M", \
#                                  when(col("Monetary") >= M[2] , 4).\
#                                  when(col("Monetary") >= M[1] , 3).\
#                                  when(col("Monetary") >= M[0] , 2).\
#                                  otherwise(1))

# rfm_table = rfm_table.withColumn("RFM_Score", concat(col("R"), col("F"), col("M")))

In [46]:
F

[1.0, 1.0, 2.0]

In [ ]:
grouped_by_rfmscore = rfm_table.groupBy("RFM_Score").count().orderBy("count", ascending=False)
#converting data to pandas
grouped_by_rfmscore_pandas = grouped_by_rfmscore.toPandas()


In [ ]:
grouped_by_rfmscore_pandas.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

a4_dims = (20, 9)
fig, ax = plt.subplots(figsize=a4_dims)
ax = sns.barplot(x='RFM_Score', y='count', data=grouped_by_rfmscore_pandas,saturation=7)
ax.set(xlabel='RFM_Score', ylabel='Number of Customer')
plt.show()

##  Below are some segments:
1. High-spending New Customers — (4–1–3 and 4–1–4).These are customers who transacted only once, but very recently and they spent a lot.
2. Lowest-Spending Active Loyal Customers.(3–4–1 and 4–4–1). They transacted recently and do so often, but spend the least.
3. Churned Best Customers.(1–3–3, 1–4–3, 1–3–4 and 1–4–4). They transacted frequently and spent a lot, but it’s been a long time since they’ve transacted.

## LTV

In [ ]:
cluster_schema = StructType([ 
     StructField('CustomerId', DoubleType(), True),
    StructField('R', IntegerType(), True),
    StructField('F', IntegerType(), True),
    StructField('M', IntegerType(), True)
    ])

user_schema = StructType([ 
     StructField('CustomerId', DoubleType(), True)
    ])

spend_schema = StructType([ 
     StructField('CustomerId', DoubleType(), True),
     StructField('Monetary', DoubleType(), True)
    ])

# Load R,F,M data for shoppers in the train set
cluster_data = rfm_table.select([c for c in rfm_table.columns if c in ['CustomerId','R','F','M']])
cluster_data.createOrReplaceTempView("train")
# Load users who have purchased in the test set
user_data = rfm_table.select([c for c in rfm_table.columns if c in ['CustomerId']])
user_data.createOrReplaceTempView("test")
# Load avg spend (in the time period) of shoppers
spend_data = rfm_table.select([c for c in rfm_table.columns if c in ['CustomerId','Monetary']])
spend_data.createOrReplaceTempView("spend")

In [ ]:
q = '''
    select DISTINCT A.*, 
       CASE WHEN B.CustomerId IS NOT NULL
       THEN 1
       ELSE 0
       END Buy
    from train A
    left join test B
    on A.CustomerId = B.CustomerId
'''
final_df = spark.sql(q)
final_df.createOrReplaceTempView("final_df")
final_df.show(5)

In [ ]:
# Get total number of shoppers
total_shoppers_q = '''
SELECT COUNT(*) FROM final_df
'''
spark.sql(total_shoppers_q).show()

In [ ]:
R_df = spark.sql('''
SELECT ROUND(COUNT(*)/457958,4) as pct, R FROM final_df WHERE Buy == 1 GROUP BY R''')
F_df = spark.sql('''
SELECT ROUND(COUNT(*)/457958,4) as pct, F FROM final_df WHERE Buy == 1 GROUP BY F''')
M_df = spark.sql('''
SELECT ROUND(COUNT(*)/457958,4) as pct, M FROM final_df WHERE Buy == 1 GROUP BY M''')
# Let's look at what the M dataframe looks like
M_df.show(5)

In [ ]:
# Convert to pandas for plotting sake
R_df.toPandas().plot(x="R", y="pct", kind='scatter')

In [ ]:
# Convert to pandas for plotting sake
F_df.toPandas().plot(x="F", y="pct", kind='scatter')

In [ ]:
# Convert to pandas for plotting sake
M_df.toPandas().plot(x="M", y="pct", kind='scatter')

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.pipeline import Pipeline

# Data
data = final_df.withColumnRenamed("Buy", "label")
training_set, testing_set = data.randomSplit([0.75,0.25])

# Model and pipeline
lr = LogisticRegression()
assembler = VectorAssembler(inputCols = ["R","F","M"], outputCol="features")
pipeline = Pipeline(stages=[assembler,lr])

# Fit the pipeline
fitted_pipeline = pipeline.fit(training_set)

# Get model from the fitted pipeline
lrModel = fitted_pipeline.stages[-1]

In [ ]:
# Get predictions from the model
# The probability value is their retention rate
predictions = lrModel.transform(assembler.transform(testing_set)).select("CustomerId", "probability")
predictions.show(5)

In [ ]:
secondelement=udf(lambda v:float(v[1]),FloatType())
predictions = (predictions.select(secondelement('probability'), "CustomerId")
        .withColumnRenamed("<lambda>(probability)", "retention_rate"))
predictions.show(50)

In [ ]:
data = predictions.join(spend_data, how="inner", on ="CustomerId")
data.show(5)

In [ ]:
def calc_clv(df, num_periods, discount):

    clv = lambda num_periods, retention, spend, discount: sum(
        [spend*(retention/(1 + discount))**t for t in range(1, num_periods + 1)])
    clv_rdd = (df.rdd.map(lambda x: (x["CustomerId"],
                                     round(clv(num_periods, x["retention_rate"],
                                           x["Monetary"], discount), 2))))

    clv_df = clv_rdd.toDF().withColumnRenamed("_1", "CustomerId").withColumnRenamed("_2", "CLV")

    return clv_df

In [ ]:
lifetime = 10
discount = 0.1
clv_df = calc_clv(data, 10, 0.1) 
clv_df.show(5)